In [25]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import requests                  
from bs4 import BeautifulSoup    
import re                        
from datetime import datetime    
from time import mktime          

In [26]:
##variables
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-summary"
token = "22990f9b4amsh4df04e987b36e1bp18331cjsn862ed02f8065"
day_begin = "01-01-2009"
day_end = "01-01-2019"
interval = "1d"

In [3]:
#START THE PROCESS
def start():
    Stockslist = list()
    Stocknames = list()
    End_of_Party = 0
    while End_of_Party == 0:
        input_stock = (input("Insert the Name of the Company to add stock, otherwise enter \"done\": "))
        input_stock = input_stock.upper()
        if input_stock == "DONE":
            End_of_Party += 1
        else:
            Stocknames.append(input_stock)
            input_stock = (input("Insert the SHORT NAME (Stockname) to add stock, otherwise enter \"done\": "))
            input_stock = input_stock.upper()
            if input_stock == "DONE":
                End_of_Party += 1
            else:
                Stockslist.append(input_stock)
            continue
        return Stocknames, Stockslist

In [4]:
#activate the load data function
def get_data_rollin(Stocknames, Shortnames):
    index = 0
    oop = {}
    for names in Shortnames:
        Stockname = Stocknames[index]
        index +=1
        oop[Stockname] = get_info(names)
    return oop
    
    
def get_info(Stock):
    Stock = load_csv_data(Stock, interval, day_begin, day_end)
    return Stock

In [5]:
def _get_crumbs_and_cookies(stock):
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [6]:
def convert_to_unix(date):
    
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [7]:
def load_csv_data(stock, interval, day_begin, day_end):
  
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        if len(website.text) < 1000:
            print("something with the data of",stock,"went wrong")
        else: print("mhm we've just received a shit ton of data from",stock)
        
        return website.text.split('\n')[:-1]
    
    

In [8]:
def delete_wrong_datasets(List_of_raws, compname):
    index = 0 
    companynames = list()
    for x in list(List_of_raws.values()):
        if len(List_of_raws[compname[index]]) < 200:
            #print(len(List_of_raws[compname[index]]))
            del List_of_raws[compname[index]]
            index += 1
        else:
            companynames.append(compname[index])
            index += 1
    return List_of_raws
            
            

In [9]:
def new_comp_names(List_of_raws, compnames):
    index = 0 
    updated_compnames = list()
    for x in List_of_raws:
        if x in compnames:
            updated_compnames.append(x)
    return updated_compnames
            

In [10]:
def getting_data_seperated(List_of_raws, compname):
    list2 = list()
    compnameindex = -1
    for x in List_of_raws.values():
        compnameindex += 1  
        temp = list()
        for z in x:
            temp.append(z.split(","))
            new_cols = ["Date"] 
        for name in temp[0]:
            if name != "Date":
                name = compname[compnameindex] + name
                new_cols.append(name)
        temp[0] = new_cols
        list2.append(temp)
        
    list2 = creating_dfs(Startprocess[0], list2)
    
    return list2
#df2 = pd.DataFrame(list2)
#df2.transpose()

In [46]:
def creating_dfs(Compnames, List):
    df_all = pd.DataFrame(List[0])
    df_all = df_all.rename(columns=df_all.iloc[0])
    df_all = df_all.drop(0,axis=0)
    index = 1 
    while index < len(List):
        df = pd.DataFrame(List[index]) 
        df = df.rename(columns=df.iloc[0])
        df = df.drop(0,axis=0)
        df_all = pd.merge(df_all,df, on="Date") 
        index += 1

    return df_all

In [47]:
Startprocess = start()
List_of_raws = get_data_rollin(Startprocess[0],Startprocess[1])
List_of_raws = delete_wrong_datasets(List_of_raws, Startprocess[0])
New_compnamelist = new_comp_names(List_of_raws, Startprocess[0])
df_all = getting_data_seperated(List_of_raws, New_compnamelist)

df_all

#SAP = SAP // APPLE = AAPL // AMAZON = AMZN // ADIDAS = ADDYY // STARBUCKS = SBUX // FACEBOOK = FB  
#WALLMART = WMT // JP MORGAN =JPM // MICROSOFT = MSFT 

Insert the Name of the Company to add stock, otherwise enter "done":  aapl
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  aapl
Insert the Name of the Company to add stock, otherwise enter "done":  sap
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  sap
Insert the Name of the Company to add stock, otherwise enter "done":  fb
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  fb
Insert the Name of the Company to add stock, otherwise enter "done":  wmt
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  wmt
Insert the Name of the Company to add stock, otherwise enter "done":  sbux
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  sbux
Insert the Name of the Company to add stock, otherwise enter "done":  done


mhm we've just received a shit ton of data from AAPL
mhm we've just received a shit ton of data from SAP
mhm we've just received a shit ton of data from FB
mhm we've just received a shit ton of data from WMT
something with the data of SBUX went wrong


,Date,AAPLOpen,AAPLHigh,AAPLLow,AAPLClose,AAPLAdj Close,AAPLVolume,SAPOpen,SAPHigh,SAPLow,...,FBLow,FBClose,FBAdj Close,FBVolume,WMTOpen,WMTHigh,WMTLow,WMTClose,WMTAdj Close,WMTVolume
0,2012-05-18,76.279999,77.629997,74.597145,75.768570,66.375458,183073100,59.959999,59.970001,58.349998,...,38.000000,38.230000,38.230000,573576400,61.869999,62.630001,61.759998,62.430000,52.374886,16630700
1,2012-05-21,76.357140,80.220001,76.292854,80.182854,70.242508,157776500,58.340000,58.950001,57.900002,...,33.000000,34.029999,34.029999,168192700,62.380001,63.099998,62.139999,63.040001,52.886631,11149900
2,2012-05-22,81.364288,81.982857,78.940002,79.567146,69.703125,173717600,59.410000,59.759998,58.209999,...,30.940001,31.000000,31.000000,101786600,62.990002,63.950001,62.900002,63.730000,53.465492,12074400
3,2012-05-23,79.642860,81.828575,79.032860,81.508568,71.403862,146224400,58.529999,58.970001,57.740002,...,31.360001,32.000000,32.000000,73600000,63.389999,64.690002,63.290001,64.580002,54.178593,15682200
4,2012-05-24,82.267143,82.357140,80.175713,80.760002,70.748108,124057500,58.520000,58.599998,57.430000,...,31.770000,33.029999,33.029999,50237200,64.709999,65.180000,64.459999,65.070000,54.589672,11302700
5,2012-05-25,80.655716,80.835716,79.781425,80.327141,70.368912,82126800,57.080002,57.619999,57.009998,...,31.110001,31.910000,31.910000,37149800,64.860001,65.500000,64.800003,65.309998,54.791016,10179900
6,2012-05-29,81.557144,82.000000,80.758568,81.752853,71.617897,95127200,58.730000,58.980000,57.820000,...,28.650000,28.840000,28.840000,78063400,65.400002,65.760002,65.209999,65.680000,55.101418,10036900
7,2012-05-30,81.314285,82.855713,80.937141,82.738571,72.481392,132357400,57.130001,57.250000,56.650002,...,27.860001,28.190001,28.190001,57267900,65.410004,65.949997,65.330002,65.440002,54.900082,9979700
8,2012-05-31,82.962860,83.071426,81.637146,82.532860,72.301178,122918600,58.049999,58.080002,56.660000,...,26.830000,29.600000,29.600000,111639200,65.400002,66.660004,65.379997,65.820000,55.218868,16765500
9,2012-06-01,81.308571,81.807144,80.074287,80.141426,70.206215,130246900,55.689999,56.130001,54.970001,...,27.389999,27.719999,27.719999,41855500,65.430000,65.879997,65.129997,65.550003,54.992363,15300700
